In [ ]:
!pip install jupyter-dash dash==2.12.1 plotly pandas openpyxl pyngrok


In [ ]:
!pip install plotly
!pip install jupyter-dash dash==2.12.1
!pip install jupyter-dash pyngrok
# Import libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import Dash
from dash import dcc, html

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Cleaned_Wuzzuf.xlsx to Cleaned_Wuzzuf (1).xlsx


In [ ]:
# Load the Excel file

df = pd.read_excel(r'Cleaned_Wuzzuf.xlsx')

# Custom color palette from the uploaded image
custom_color_palette = [
    "#12181E", "#171F27", "#1E2832", "#263441", "#2A3947", "#55616C",
    "#707A84", "#798994", "#8596A3", "#9DAABB", "#BDC2C6", "#EAEBED"
]

In [ ]:



# # Prepare data: Top 10 job titles by frequency
# top_job_titles = df['job_title'].value_counts().nlargest(10).reset_index()
# top_job_titles.columns = ['job_title', 'count']

# # Create bar chart
# fig = px.bar(
#     top_job_titles,
#     x='count',
#     y='job_title',
#     orientation='h',
#     color='job_title',
#     color_discrete_sequence=custom_color_palette,
#     title='Top 10 Job Titles by Demand'
# )

# fig.update_layout(
#     yaxis=dict(autorange="reversed"),
#     showlegend=False
# )

# fig.show()


In [ ]:
# Prepare data: Top 10 job titles by frequency
top_job_titles = df['job_title'].value_counts().nlargest(10).reset_index()
top_job_titles.columns = ['job_title', 'count']

# Create bar chart
fig = px.bar(
    top_job_titles,
    x='count',
    y='job_title',
    orientation='h',
    color='job_title',
    color_discrete_sequence=custom_color_palette,
    title='Top 10 Job Titles by Demand'
)

fig.update_layout(
    yaxis=dict(categoryorder='total ascending'),  # this reverses because bars are horizontal (x = count)
    showlegend= True
)

fig.show()

In [ ]:
# city_distribution = df['city'].value_counts().nlargest(10).reset_index()
# city_distribution.columns = ['city', 'count']

# fig2 = px.pie(
#     city_distribution,
#     names='city',
#     values='count',
#     title='Job Distribution by City',
#     color_discrete_sequence=custom_color_palette
# )
# fig2.show()


In [ ]:
# Step 1: Get the full city/job count
city_distribution = df['city'].value_counts().reset_index()
city_distribution.columns = ['city', 'count']

# Step 2: Add coordinates manually
egypt_city_coords = pd.DataFrame({
    'city': ["Cairo", "Giza", "Alexandria", "Port Said", "Suez", "Damietta", "Dakahlia", "Sharkia", "Qalyubia", "Kafr El Sheikh",
             "Beheira", "Ismailia", "Gharbia", "Monufia", "Faiyum", "Beni Suef", "Minya", "Assiut", "Sohag", "Qena", "Luxor",
             "Aswan", "Red Sea", "New Valley", "Matrouh", "North Sinai", "South Sinai"],
    'lat': [30.0444, 30.0131, 31.2001, 31.2653, 29.9668, 31.4165, 31.0364, 30.7320, 30.3339, 31.3085,
            30.8480, 30.6043, 30.8754, 30.5972, 29.3084, 29.0661, 28.1099, 27.1800, 26.5560, 26.1551,
            25.6872, 24.0889, 27.2579, 25.3250, 31.3525, 30.3060, 28.1099],
    'lon': [31.2357, 31.2089, 29.9187, 32.3019, 32.5498, 31.8100, 31.3807, 31.7195, 31.2421, 30.9336,
            30.3400, 32.2723, 30.8195, 30.9876, 30.8467, 31.0978, 30.7503, 31.1837, 31.6948, 32.7160,
            32.6396, 32.8998, 33.8129, 30.5467, 27.2289, 33.7992, 33.3000]
})

# Step 3: Merge data
city_distribution = city_distribution.merge(egypt_city_coords, on='city', how='left')

# Step 4: Plot
fig = px.scatter_mapbox(
    city_distribution,
    lat="lat",
    lon="lon",
    size="count",
    color="count",
    hover_name="city",
    size_max=40,
    zoom=5,
    mapbox_style="carto-positron",
    title="Job Distribution by City in Egypt",
    color_discrete_sequence=custom_color_palette
)

fig.show()

In [ ]:
# Fill missing annual salaries with 0
df['annual_salary'] = df['annual_salary'].fillna(0)

top_titles_salary = df['job_title'].value_counts().nlargest(10).index
avg_salary = df[df['job_title'].isin(top_titles_salary)] \
    .groupby('job_title')['annual_salary'] \
    .mean().sort_values(ascending=False).reset_index()

fig3 = px.bar(
    avg_salary,
    x='annual_salary',
    y='job_title',
    orientation='h',
    color='job_title',
    color_discrete_sequence=custom_color_palette,
    title='Average Annual Salary by Job Title'
)
fig3.update_layout(
    yaxis=dict(categoryorder='total ascending'),  # this reverses because bars are horizontal (x = count)
    showlegend= True
)
fig3.show()


In [ ]:
df['post_date'] = pd.to_datetime(df['post_date'])
df['month'] = df['post_date'].dt.to_period('M').astype(str)

monthly_trend = df.groupby('month').size().reset_index(name='job_posts')

fig4 = px.line(
    monthly_trend,
    x='month',
    y='job_posts',
    title='Job Post Trends Over Time',
    markers=True,
    color_discrete_sequence=[custom_color_palette[4]]
)
fig4.update_xaxes(tickangle=45)
fig4.show()


In [ ]:
# fig5 = px.box(
#     df[df['annual_salary'] > 0],
#     x='job_category',
#     y='annual_salary',
#     title='Salary vs. Job Category',
#     color='job_category',
#     color_discrete_sequence=custom_color_palette
# )
# fig5.update_layout(xaxis_tickangle=-45)
# fig5.show()


In [ ]:
# # Process top_skills (comma-separated)
# df['top_skills'] = df['top_skills'].fillna('')
# skills_exploded = df[['year', 'top_skills']].copy()
# skills_exploded['top_skills'] = skills_exploded['top_skills'].str.split(',')
# skills_exploded = skills_exploded.explode('top_skills')
# skills_exploded['top_skills'] = skills_exploded['top_skills'].str.strip()

# # Count top 10 skills per year
# top_skills_yearly = skills_exploded.groupby(['year', 'top_skills']) \
#     .size().reset_index(name='count')

# top_skills_per_year = top_skills_yearly.groupby('year') \
#     .apply(lambda x: x.nlargest(10, 'count')).reset_index(drop=True)

# fig6 = px.bar(
#     top_skills_per_year,
#     x='top_skills',
#     y='count',
#     color='year',
#     barmode='group',
#     title='Top Skills in Demand by Year',
#     color_discrete_sequence=custom_color_palette
# )
# fig6.update_layout(xaxis_tickangle=45)
# fig6.show()


In [ ]:
# Group by job_category and sum the number of vacancies
vacancies_by_category = df.groupby('Category')['num_vacancies'].sum().sort_values(ascending=False).reset_index()

# Plot with Plotly
fig5 = px.bar(
    vacancies_by_category,
    x= 'Category',
    y= 'num_vacancies',
    color='Category',
    color_discrete_sequence=custom_color_palette,
    title='Number of Vacancies by Job Category'
)

fig.update_layout(
    xaxis=dict(categoryorder='total ascending'),  # this reverses because bars are horizontal (x = count)
    showlegend= True
)

fig5.show()

In [ ]:
# Metric
total_postings = len(df)

# Color
primary_color = "#263441"

# Indicator card
fig = go.Figure(go.Indicator(
    mode="number",
    value=total_postings,
    title={"text": "Total Job Postings"},
    number={'font': {'size': 40, 'color': primary_color}},
    title_font={'size': 20, 'color': primary_color},
    domain={'x': [0, 1], 'y': [0, 1]}
))

fig.update_layout(paper_bgcolor="#EAEBED")
fig.show()

In [ ]:

# Metric
total_vacancies = df['num_vacancies'].sum()

# Color
primary_color = "#263441"

# Indicator card
fig = go.Figure(go.Indicator(
    mode="number",
    value=total_vacancies,
    title={"text": "Total Vacancies"},
    number={'font': {'size': 40, 'color': primary_color}},
    title_font={'size': 20, 'color': primary_color},
    domain={'x': [0, 1], 'y': [0, 1]}
))

fig.update_layout(paper_bgcolor="#EAEBED")
fig.show()

In [ ]:
# # ====================
# # Define your palette
# # ====================
# custom_color_palette = [
#     "#12181E", "#171F27", "#1E2832", "#263441", "#2A3947", "#55616C",
#     "#707A84", "#798994", "#8596A3", "#9DAABB", "#BDC2C6", "#EAEBED"
# ]
# primary_color = "#263441"

# # ============
# # 1. Top jobs
# # ============
# top_job_titles = df['job_title'].value_counts().nlargest(10).reset_index()
# top_job_titles.columns = ['job_title', 'count']
# fig1 = px.bar(
#     top_job_titles,
#     x='count',
#     y='job_title',
#     orientation='h',
#     color='job_title',
#     color_discrete_sequence=custom_color_palette,
#     title='Top 10 Job Titles by Demand'
# )
# fig1.update_layout(yaxis=dict(categoryorder='total ascending'), showlegend=True)

# # ==========================
# # 2. Job Distribution Map
# # ==========================
# city_distribution = df['city'].value_counts().reset_index()
# city_distribution.columns = ['city', 'count']
# egypt_city_coords = pd.DataFrame({
#     'city': ["Cairo", "Giza", "Alexandria", "Port Said", "Suez", "Damietta", "Dakahlia", "Sharkia", "Qalyubia", "Kafr El Sheikh",
#              "Beheira", "Ismailia", "Gharbia", "Monufia", "Faiyum", "Beni Suef", "Minya", "Assiut", "Sohag", "Qena", "Luxor",
#              "Aswan", "Red Sea", "New Valley", "Matrouh", "North Sinai", "South Sinai"],
#     'lat': [30.0444, 30.0131, 31.2001, 31.2653, 29.9668, 31.4165, 31.0364, 30.7320, 30.3339, 31.3085,
#             30.8480, 30.6043, 30.8754, 30.5972, 29.3084, 29.0661, 28.1099, 27.1800, 26.5560, 26.1551,
#             25.6872, 24.0889, 27.2579, 25.3250, 31.3525, 30.3060, 28.1099],
#     'lon': [31.2357, 31.2089, 29.9187, 32.3019, 32.5498, 31.8100, 31.3807, 31.7195, 31.2421, 30.9336,
#             30.3400, 32.2723, 30.8195, 30.9876, 30.8467, 31.0978, 30.7503, 31.1837, 31.6948, 32.7160,
#             32.6396, 32.8998, 33.8129, 30.5467, 27.2289, 33.7992, 33.3000]
# })
# city_distribution = city_distribution.merge(egypt_city_coords, on='city', how='left')
# fig2 = px.scatter_mapbox(
#     city_distribution,
#     lat="lat",
#     lon="lon",
#     size="count",
#     color="count",
#     hover_name="city",
#     size_max=40,
#     zoom=5,
#     mapbox_style="carto-positron",
#     title="Job Distribution by City in Egypt"
# )

# # ==========================
# # 3. Average Salary by Job
# # ==========================
# df['annual_salary'] = df['annual_salary'].fillna(0)
# top_titles_salary = df['job_title'].value_counts().nlargest(10).index
# avg_salary = df[df['job_title'].isin(top_titles_salary)] \
#     .groupby('job_title')['annual_salary'] \
#     .mean().sort_values(ascending=False).reset_index()
# fig3 = px.bar(
#     avg_salary,
#     x='annual_salary',
#     y='job_title',
#     orientation='h',
#     color='job_title',
#     color_discrete_sequence=custom_color_palette,
#     title='Average Annual Salary by Job Title'
# )
# fig3.update_layout(yaxis=dict(categoryorder='total ascending'), showlegend=True)

# # ========================
# # 4. Monthly Job Trends
# # ========================
# df['post_date'] = pd.to_datetime(df['post_date'])
# df['month'] = df['post_date'].dt.to_period('M').astype(str)
# monthly_trend = df.groupby('month').size().reset_index(name='job_posts')
# fig4 = px.line(
#     monthly_trend,
#     x='month',
#     y='job_posts',
#     title='Job Post Trends Over Time',
#     markers=True,
#     color_discrete_sequence=[custom_color_palette[4]]
# )
# fig4.update_xaxes(tickangle=45)

# # ========================
# # 5. Vacancies by Category
# # ========================
# vacancies_by_category = df.groupby('Category')['num_vacancies'].sum().sort_values(ascending=False).reset_index()
# fig5 = px.bar(
#     vacancies_by_category,
#     x='Category',
#     y='num_vacancies',
#     color='Category',
#     color_discrete_sequence=custom_color_palette,
#     title='Number of Vacancies by Job Category'
# )

# # ========================
# # 6. Metrics (Total Posts & Vacancies)
# # ========================
# fig6 = go.Figure(go.Indicator(
#     mode="number",
#     value=len(df),
#     title={"text": "Total Job Postings"},
#     number={'font': {'size': 40, 'color': primary_color}},
#     title_font={'size': 20, 'color': primary_color},
#     domain={'x': [0, 1], 'y': [0, 1]}
# ))
# fig6.update_layout(paper_bgcolor="#EAEBED")

# fig7 = go.Figure(go.Indicator(
#     mode="number",
#     value=df['num_vacancies'].sum(),
#     title={"text": "Total Vacancies"},
#     number={'font': {'size': 40, 'color': primary_color}},
#     title_font={'size': 20, 'color': primary_color},
#     domain={'x': [0, 1], 'y': [0, 1]}
# ))
# fig7.update_layout(paper_bgcolor="#EAEBED")

# # ======================
# # Create Dash App Layout
# # ======================
# app = JupyterDash(__name__)
# app.layout = html.Div([
#     html.H1("Egypt Job Market Dashboard", style={'textAlign': 'center'}),

#     html.Div([
#         html.Div([dcc.Graph(figure=fig6)], className='six columns'),
#         html.Div([dcc.Graph(figure=fig7)], className='six columns'),
#     ], className='row'),

#     dcc.Graph(figure=fig1),
#     dcc.Graph(figure=fig3),
#     dcc.Graph(figure=fig5),
#     dcc.Graph(figure=fig2),
#     dcc.Graph(figure=fig4)
# ], style={'padding': '20px'})

# # ======================
# # Run the app inline
# # ======================
# app.run_server(mode='inline')


In [ ]:
# # Custom color palette
# custom_color_palette = [
#     "#12181E", "#171F27", "#1E2832", "#263441", "#2A3947", "#55616C",
#     "#707A84", "#798994", "#8596A3", "#9DAABB", "#BDC2C6", "#EAEBED"
# ]

# # 1. Top 10 Job Titles by Frequency
# job_title_counts = df['job_title'].value_counts().nlargest(10).reset_index()
# job_title_counts.columns = ['job_title', 'count']
# fig1 = px.bar(job_title_counts, x='count', y='job_title', orientation='h', color='job_title', color_discrete_sequence=custom_color_palette, title='Top 10 Job Titles by Demand')
# fig1.update_layout(yaxis=dict(categoryorder='total ascending'))

# # 2. Job Distribution by City Map
# city_counts = df['city'].value_counts().reset_index()
# city_counts.columns = ['city', 'count']
# city_coords = pd.DataFrame({
#     'city': ["Cairo", "Giza", "Alexandria", "Port Said", "Suez", "Damietta", "Dakahlia", "Sharkia", "Qalyubia", "Kafr El Sheikh", "Beheira", "Ismailia", "Gharbia", "Monufia", "Faiyum", "Beni Suef", "Minya", "Assiut", "Sohag", "Qena", "Luxor", "Aswan", "Red Sea", "New Valley", "Matrouh", "North Sinai", "South Sinai"],
#     'lat': [30.0444, 30.0131, 31.2001, 31.2653, 29.9668, 31.4165, 31.0364, 30.7320, 30.3339, 31.3085, 30.8480, 30.6043, 30.8754, 30.5972, 29.3084, 29.0661, 28.1099, 27.1800, 26.5560, 26.1551, 25.6872, 24.0889, 27.2579, 25.3250, 31.3525, 30.3060, 28.1099],
#     'lon': [31.2357, 31.2089, 29.9187, 32.3019, 32.5498, 31.8100, 31.3807, 31.7195, 31.2421, 30.9336, 30.3400, 32.2723, 30.8195, 30.9876, 30.8467, 31.0978, 30.7503, 31.1837, 31.6948, 32.7160, 32.6396, 32.8998, 33.8129, 30.5467, 27.2289, 33.7992, 33.3000]
# })
# city_map_df = city_counts.merge(city_coords, on='city', how='left')
# fig2 = px.scatter_mapbox(city_map_df, lat='lat', lon='lon', size='count', color='count', hover_name='city', size_max=40, zoom=5, mapbox_style='carto-positron', title='Job Distribution by City')

# # 3. Average Salary by Top 10 Job Titles
# df['annual_salary'] = df['annual_salary'].fillna(0)
# top_titles_salary = df['job_title'].value_counts().nlargest(10).index
# avg_salary = df[df['job_title'].isin(top_titles_salary)].groupby('job_title')['annual_salary'].mean().sort_values(ascending=False).reset_index()
# fig3 = px.bar(avg_salary, x='annual_salary', y='job_title', orientation='h', color='job_title', color_discrete_sequence=custom_color_palette, title='Average Annual Salary by Job Title')
# fig3.update_layout(yaxis=dict(categoryorder='total ascending'))

# # 4. Job Post Trends Over Time
# df['post_date'] = pd.to_datetime(df['post_date'])
# df['month'] = df['post_date'].dt.to_period('M').astype(str)
# monthly_trend = df.groupby('month').size().reset_index(name='job_posts')
# fig4 = px.line(monthly_trend, x='month', y='job_posts', title='Job Post Trends Over Time', markers=True, color_discrete_sequence=[custom_color_palette[4]])
# fig4.update_xaxes(tickangle=45)

# # 5. Vacancies by Category
# vacancies_by_category = df.groupby('Category')['num_vacancies'].sum().sort_values(ascending=False).reset_index()
# fig5 = px.bar(vacancies_by_category, x='Category', y='num_vacancies', color='Category', color_discrete_sequence=custom_color_palette, title='Number of Vacancies by Job Category')

# # 6. Metric: Total Postings
# fig6 = go.Figure(go.Indicator(mode="number", value=len(df), title={"text": "Total Job Postings"}, number={'font': {'size': 40}}, domain={'x': [0, 1], 'y': [0, 1]}))

# # 7. Metric: Total Vacancies
# fig7 = go.Figure(go.Indicator(mode="number", value=df['num_vacancies'].sum(), title={"text": "Total Vacancies"}, number={'font': {'size': 40}}, domain={'x': [0, 1], 'y': [0, 1]}))

# # Build the dashboard
# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# app = Dash(__name__, external_stylesheets=external_stylesheets)

# app.layout = html.Div([
#     html.H1("Hire+ Market Dashboard", style={'textAlign': 'center'}),
#     html.Div([
#         html.Div([dcc.Graph(figure=fig6)], className='six columns'),
#         html.Div([dcc.Graph(figure=fig7)], className='six columns')
#     ], className='row'),

#     html.Div([
#         html.Div([dcc.Graph(figure=fig1)], className='six columns'),
#         html.Div([dcc.Graph(figure=fig3)], className='six columns')
#     ], className='row'),

#     html.Div([
#         html.Div([dcc.Graph(figure=fig4)], className='six columns'),
#         html.Div([dcc.Graph(figure=fig5)], className='six columns')
#     ], className='row'),

#     html.Div([
#         html.Div([dcc.Graph(figure=fig2)], className='twelve columns')
#     ], className='row')
# ])

# # Run the app
# from pyngrok import ngrok

# # Run Dash app on a known port
# port = 8050
# #ngrok.set_auth_token("your_ngrok_auth_token")  # Optional if you have an auth token
# #public_url = ngrok.connect(port)
# #print("Dashboard URL:", public_url)

# app.run_server(port=port)


In [ ]:

# Custom colors
custom_color_palette = [
    "#12181E", "#171F27", "#1E2832", "#263441", "#2A3947", "#55616C",
    "#707A84", "#798994", "#8596A3", "#9DAABB", "#BDC2C6", "#EAEBED"
]

### VISUALIZATION 1: Top Job Titles ###
top_job_titles = df['job_title'].value_counts().nlargest(10).reset_index()
top_job_titles.columns = ['job_title', 'count']
fig1 = px.bar(
    top_job_titles, x='count', y='job_title', orientation='h',
    color='job_title', color_discrete_sequence=custom_color_palette,
    title='Top 10 Job Titles by Demand'
)

### VISUALIZATION 2: Job Distribution Map ###
city_distribution = df['city'].value_counts().reset_index()
city_distribution.columns = ['city', 'count']

# Coordinates
egypt_city_coords = pd.DataFrame({
    'city': ["Cairo", "Giza", "Alexandria", "Port Said", "Suez", "Damietta", "Dakahlia", "Sharkia", "Qalyubia", "Kafr El Sheikh",
             "Beheira", "Ismailia", "Gharbia", "Monufia", "Faiyum", "Beni Suef", "Minya", "Assiut", "Sohag", "Qena", "Luxor",
             "Aswan", "Red Sea", "New Valley", "Matrouh", "North Sinai", "South Sinai"],
    'lat': [30.0444, 30.0131, 31.2001, 31.2653, 29.9668, 31.4165, 31.0364, 30.7320, 30.3339, 31.3085,
            30.8480, 30.6043, 30.8754, 30.5972, 29.3084, 29.0661, 28.1099, 27.1800, 26.5560, 26.1551,
            25.6872, 24.0889, 27.2579, 25.3250, 31.3525, 30.3060, 28.1099],
    'lon': [31.2357, 31.2089, 29.9187, 32.3019, 32.5498, 31.8100, 31.3807, 31.7195, 31.2421, 30.9336,
            30.3400, 32.2723, 30.8195, 30.9876, 30.8467, 31.0978, 30.7503, 31.1837, 31.6948, 32.7160,
            32.6396, 32.8998, 33.8129, 30.5467, 27.2289, 33.7992, 33.3000]
})
city_distribution = city_distribution.merge(egypt_city_coords, on='city', how='left')
fig2 = px.scatter_mapbox(
    city_distribution, lat="lat", lon="lon", size="count", color="count",
    hover_name="city", size_max=40, zoom=5, mapbox_style="carto-positron",
    title="Job Distribution by City in Egypt"
)

### VISUALIZATION 3: Average Salary by Job Title ###
df['annual_salary'] = df['annual_salary'].fillna(0)
top_titles_salary = df['job_title'].value_counts().nlargest(10).index
avg_salary = df[df['job_title'].isin(top_titles_salary)] \
    .groupby('job_title')['annual_salary'] \
    .mean().sort_values(ascending=False).reset_index()
fig3 = px.bar(
    avg_salary, x='annual_salary', y='job_title', orientation='h',
    color='job_title', color_discrete_sequence=custom_color_palette,
    title='Average Annual Salary by Job Title'
)

### VISUALIZATION 4: Job Post Trends Over Time ###
df['post_date'] = pd.to_datetime(df['post_date'])
df['month'] = df['post_date'].dt.to_period('M').astype(str)
monthly_trend = df.groupby('month').size().reset_index(name='job_posts')
fig4 = px.line(
    monthly_trend, x='month', y='job_posts', markers=True,
    color_discrete_sequence=[custom_color_palette[4]],
    title='Job Post Trends Over Time'
)

### VISUALIZATION 5: Number of Vacancies by Category ###
vacancies_by_category = df.groupby('Category')['num_vacancies'].sum().sort_values(ascending=False).reset_index()
fig5 = px.bar(
    vacancies_by_category, x='Category', y='num_vacancies',
    color='Category', color_discrete_sequence=custom_color_palette,
    title='Number of Vacancies by Job Category'
)

### METRICS ###
total_postings = len(df)
total_vacancies = df['num_vacancies'].sum()

fig6 = go.Figure(go.Indicator(
    mode="number", value=total_postings,
    title={"text": "Total Job Postings"},
    number={'font': {'size': 40, 'color': "#263441"}},
))
fig6.update_layout(paper_bgcolor="#EAEBED")

fig7 = go.Figure(go.Indicator(
    mode="number", value=total_vacancies,
    title={"text": "Total Vacancies"},
    number={'font': {'size': 40, 'color': "#263441"}},
))
fig7.update_layout(paper_bgcolor="#EAEBED")

### DASH LAYOUT ###
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Hire+ Job Market Dashboard"),

    dcc.Graph(figure=fig1),
    dcc.Graph(figure=fig2),
    dcc.Graph(figure=fig3),
    dcc.Graph(figure=fig4),
    dcc.Graph(figure=fig5),
     html.Div([
        html.Div(dcc.Graph(figure=fig6), style={'display': 'inline-block', 'width': '50%'}),
        html.Div(dcc.Graph(figure=fig7), style={'display': 'inline-block', 'width': '50%'}),
    ]),

])

if __name__ == '__main__':
    app.run_server(debug=True, port=8050)

<IPython.core.display.Javascript object>